<a href="https://colab.research.google.com/github/kouib123/Creating-an-Incremental-Loading-Versus-Flush-and-Fill-on-SQL-Server-For-ETL-Processing/blob/master/Auto_MPG_classic_regression_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*The Auto MPG dataset is a classic regression problem in which the goal is to predict a continuous numeric value, namely the miles per gallon (mpg) for a given car.*

In [43]:
import pandas as pd
import numpy as np
import requests

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
response = requests.get(url)

# Write the contents of the response to a file
with open('auto-mpg.data', 'w') as f:
    f.write(response.text)

# Read the file into a DataFrame
df = pd.read_csv('auto-mpg.data', delim_whitespace=True, header=None, names=['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 
                                                                             'origin', 'car_name'])

print(df.head(10))

    mpg  cylinders  displacement horsepower  weight  acceleration  model_year  \
0  18.0          8         307.0      130.0  3504.0          12.0          70   
1  15.0          8         350.0      165.0  3693.0          11.5          70   
2  18.0          8         318.0      150.0  3436.0          11.0          70   
3  16.0          8         304.0      150.0  3433.0          12.0          70   
4  17.0          8         302.0      140.0  3449.0          10.5          70   
5  15.0          8         429.0      198.0  4341.0          10.0          70   
6  14.0          8         454.0      220.0  4354.0           9.0          70   
7  14.0          8         440.0      215.0  4312.0           8.5          70   
8  14.0          8         455.0      225.0  4425.0          10.0          70   
9  15.0          8         390.0      190.0  3850.0           8.5          70   

   origin                   car_name  
0       1  chevrolet chevelle malibu  
1       1          buick skyla

In [44]:
# Check for missing values:
# Drop null rows
df.dropna(axis=0, inplace=True)
print("DataFrame shape after dropping null rows:", df.shape)

DataFrame shape after dropping null rows: (398, 9)


In [45]:
# Check the statistical data
print(df.describe())

,mpg,cylinders,displacement,weight,acceleration,model_year,origin
count,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,2970.424623,15.568090,76.010050,1.572864
std,7.815984,1.701004,104.269838,846.841774,2.757689,3.697627,0.802055
min,9.000000,3.000000,68.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.500000,4.000000,104.250000,2223.750000,13.825000,73.000000,1.000000
50%,23.000000,4.000000,148.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,262.000000,3608.000000,17.175000,79.000000,2.000000
max,46.600000,8.000000,455.000000,5140.000000,24.800000,82.000000,3.000000


In [46]:
# Compute pairwise correlations between columns
correlation_mtx = df.corr()

print(correlation_mtx)

                   mpg  cylinders  displacement    weight  acceleration  \
mpg           1.000000  -0.775396     -0.804203 -0.831741      0.420289   
cylinders    -0.775396   1.000000      0.950721  0.896017     -0.505419   
displacement -0.804203   0.950721      1.000000  0.932824     -0.543684   
weight       -0.831741   0.896017      0.932824  1.000000     -0.417457   
acceleration  0.420289  -0.505419     -0.543684 -0.417457      1.000000   
model_year    0.579267  -0.348746     -0.370164 -0.306564      0.288137   
origin        0.563450  -0.562543     -0.609409 -0.581024      0.205873   

              model_year    origin  
mpg             0.579267  0.563450  
cylinders      -0.348746 -0.562543  
displacement   -0.370164 -0.609409  
weight         -0.306564 -0.581024  
acceleration    0.288137  0.205873  
model_year      1.000000  0.180662  
origin          0.180662  1.000000  


Correlation
Definition 0.1. Correlation analysis measures how closely related two variables are.
• A positive correlation indicates that both variables tend to increase or decrease together.
• A negative correlation indicates that as one variable increases the other tends to decrease.
• It is important not to confuse correlation with causation!
The strength of a correlations is measured between 0 and 1.
• 0.00 − 0.19 is a very weak correlation/no correlation.

> Indented block

> Indented block




• 0.2 − 0.39 a weak correlation.
• 0.4 − 0.59 is a moderate correlation.
• 0.6 − 0.79 is a strong correlation.
• 0.8 − 1 is a very strong correlation.

In [47]:
# Check Correlation analysis measures 
high_corr = correlation_mtx.abs() > 0.7
print(high_corr)

for col in high_corr.columns:
    correlated_cols = list(high_corr[col][high_corr[col]].index)
    if len(correlated_cols) > 1:
        print(f"{col} is highly correlated with {correlated_cols}")
    else:
      print(f"{col} is not correlated with {correlated_cols}")


                mpg  cylinders  displacement  weight  acceleration  \
mpg            True       True          True    True         False   
cylinders      True       True          True    True         False   
displacement   True       True          True    True         False   
weight         True       True          True    True         False   
acceleration  False      False         False   False          True   
model_year    False      False         False   False         False   
origin        False      False         False   False         False   

              model_year  origin  
mpg                False   False  
cylinders          False   False  
displacement       False   False  
weight             False   False  
acceleration       False   False  
model_year          True   False  
origin             False    True  
mpg is highly correlated with ['mpg', 'cylinders', 'displacement', 'weight']
cylinders is highly correlated with ['mpg', 'cylinders', 'displacement', 'weight']

From the result set, i noticed that i have 3 highly correlated column variables which are suggested to be cylinders, displacement and weight and our target variable is mpg column.

#** Creating a simple OLS model**

In [48]:
# Define predictors:
predictors = df[['cylinders','displacement', 'weight']]
print(predictors)

     cylinders  displacement  weight
0            8         307.0  3504.0
1            8         350.0  3693.0
2            8         318.0  3436.0
3            8         304.0  3433.0
4            8         302.0  3449.0
..         ...           ...     ...
393          4         140.0  2790.0
394          4          97.0  2130.0
395          4         135.0  2295.0
396          4         120.0  2625.0
397          4         119.0  2720.0

[398 rows x 3 columns]


In [49]:
# create and assign X to predictors 
X = predictors
print(X)

     cylinders  displacement  weight
0            8         307.0  3504.0
1            8         350.0  3693.0
2            8         318.0  3436.0
3            8         304.0  3433.0
4            8         302.0  3449.0
..         ...           ...     ...
393          4         140.0  2790.0
394          4          97.0  2130.0
395          4         135.0  2295.0
396          4         120.0  2625.0
397          4         119.0  2720.0

[398 rows x 3 columns]


In [50]:
# Define outcome:
outcome = df['mpg']
print(outcome)

0      18.0
1      15.0
2      18.0
3      16.0
4      17.0
       ... 
393    27.0
394    44.0
395    32.0
396    28.0
397    31.0
Name: mpg, Length: 398, dtype: float64


In [51]:
# Create and assign y for outcome variable
y = outcome

**Create a simple OLS Model** 

In [52]:
#Import libraries
import statsmodels.api as sm
from statsmodels.formula.api import glm
import matplotlib.pyplot as plt

# Add a constant to the predictor variables
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Print a summary of the model
print(model.summary())




                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.698
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     303.9
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          4.12e-102
Time:                        03:45:24   Log-Likelihood:                -1144.2
No. Observations:                 398   AIC:                             2296.
Df Residuals:                     394   BIC:                             2312.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           44.4472      1.482     29.989   

**Create a OLS model with Dummy variables**

In [53]:
# Create dummy variables for the 'origin' column
origin_dummies = pd.get_dummies(df['origin'], prefix='origin')

# Concatenate the original DataFrame with the dummy variables
df = pd.concat([df, origin_dummies], axis=1)

In [54]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,origin_1,origin_2,origin_3
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,1,0,0
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,1,0,0
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,1,0,0
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,1,0,0
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.00,2790.0,15.6,82,1,ford mustang gl,1,0,0
394,44.0,4,97.0,52.00,2130.0,24.6,82,2,vw pickup,0,1,0
395,32.0,4,135.0,84.00,2295.0,11.6,82,1,dodge rampage,1,0,0
396,28.0,4,120.0,79.00,2625.0,18.6,82,1,ford ranger,1,0,0


In [55]:
# Independent variables
X_dummy = df[['cylinders','displacement','weight','origin_1', 'origin_2', 'origin_3']]
X_dummy.size

2388

In [56]:
# response variable:
y_dummy = df['mpg']

In [57]:
# build ols model with dummies data:

# Add a constant to the predictor variables
X_dummy = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y_dummy, X_dummy).fit()

# Print a summary of the model
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.698
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     303.9
Date:                Sun, 26 Mar 2023   Prob (F-statistic):          4.12e-102
Time:                        03:45:37   Log-Likelihood:                -1144.2
No. Observations:                 398   AIC:                             2296.
Df Residuals:                     394   BIC:                             2312.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           44.4472      1.482     29.989   

In [59]:
# Check for multicollinearity:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
print(vif)

   VIF Factor      features
0   47.069805         const
1   10.473126     cylinders
2   15.902894  displacement
3    7.753976        weight


This code is saying that for each variable in the range of the independent variables containing the predictor variables we want to check for multicollinearity (in this case, 'var1', 'var2', 'var3', and 'var4'). The X.shape[1] expression returns the number of columns in X, which is 4 in this case.Then use the variance_inflation_factor() function from the statsmodels.stats.outliers_influence module to calculate the VIF for each variable. The VIF values are stored in a DataFrame along with the variable names, and printed to the console.

If any variables have a VIF value greater than 5 or 10 (depending on the specific context), they may be highly correlated with other variables in the model and should be investigated further.